Feature extraction and creation of datafiles for private competition DML&ML 2023.

Source of feature extraction Python code: https://www.kaggle.com/code/yunsuxiaozi/best-feature-engineer-is-all-you-need/notebook

In [7]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [6]:
pip install -U scikit-learn scipy matplotlib

     ---------------------------------------- 0.0/156.8 kB ? eta -:--:--
     -------------------------------------- 156.8/156.8 kB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.2 MB ? eta -:--:--
   - -------------------------------------- 0.4/9.2 MB 12.2 MB/s eta 0:00:01
   -- ------------------------------------- 0.6/9.2 MB 13.3 MB/s eta 0:00:01
   ----- ---------------------------------- 1.2/9.2 MB 11.0 MB/s eta 0:00:01
   ------- -------------------------------- 1.6/9.2 MB 10.4 MB/s eta 0:00:01
   -------- ------------------------------- 2.1/9.2 MB 10.1 MB/s eta 0:00:01
   ---------- ----------------------------- 2.5/9.2 MB 10.5 MB/s eta 0:00:01
   ------------ --------------------------- 2.9/9.2 MB 10.3 MB/s eta 0:00:01
   -------------- ------------------------- 3.4/9.2 MB 10.3 MB/s eta 0:00:01
   ---------------- ----------------------- 3.8/9.2 MB 10.1 MB/s eta 0:00:01
   ----------------- ---------------------- 4.1/9.2 MB 9.8 MB/s eta 0:00:01
   ------

In [8]:
#Names of input and output files
#PLEASE ADAPT THE PATHS TO YOUR LOCAL CIRCUMSTANCES!!!

#Input files, which you have to download from the public competition
train_logs_file = R"train_logs.csv"
train_scores_file = R"train_scores.csv"

#Output files, which are created for you by this script to use in the private competition
X_train_file = R"pc_X_train.csv"
y_train_file = R"pc_y_train.csv"
X_test_file = R"pc_X_test.csv"

In [9]:
def create_length_value(source_df, target_df, column_name):
    target_df[f'length_{column_name}'] = source_df[column_name].apply(lambda x:len(str(x)))

def create_mean_value(source_df, target_df, column_name):
    target_df[f'mean_{column_name}'] = source_df[column_name].groupby([source_df['id']]).mean().values

def create_std_value(source_df, target_df, column_name):
    target_df[f'std_{column_name}'] = source_df[column_name].groupby([source_df['id']]).std().values

def create_max_value(source_df, target_df, column_name):
    target_df[f'max_{column_name}'] = source_df[column_name].groupby([source_df['id']]).max().values

def create_sum_value(source_df, target_df, column_name):
    target_df[f'sum_{column_name}'] = source_df[column_name].groupby([source_df['id']]).sum().values

def create_count_value(source_df, target_df, column_name):
    target_df[f'count_{column_name}'] = source_df[column_name].groupby([source_df['id']]).count().values

In [10]:
def add_columns(source_df, target_df, column_name_1, column_name_2):
    new_column_name = f'{column_name_1}+{column_name_2}'
    target_df[new_column_name] = source_df[column_name_1]+source_df[column_name_2]
    return new_column_name

def diff_columns(source_df, target_df, column_name_1, column_name_2):
    new_column_name = f'{column_name_1}-{column_name_2}'
    target_df[new_column_name] = source_df[column_name_1]-source_df[column_name_2]
    return new_column_name

def multiply_columns(source_df, target_df, column_name_1, column_name_2):
    new_column_name = f'{column_name_1}*{column_name_2}'
    target_df[new_column_name] = source_df[column_name_1]*source_df[column_name_2]
    return new_column_name

def div_columns(source_df, target_df, column_name_1, column_name_2):
    new_column_name = f'{column_name_1}/{column_name_2}'
    target_df[new_column_name] = source_df[column_name_1]/source_df[column_name_2]
    return new_column_name

def concat_columns(source_df, target_df, columns, new_column_name=''):
    if not new_column_name:
      new_column_name = '_'.join(columns)
    target_df[new_column_name] = sum([source_df[column] for column in columns])

In [11]:
#Function for feature extraction
def deal_df(df):
    # event_id of no use,drop.
    df.drop(['event_id'],axis=1,inplace=True)

    # id
    id=df['id'].unique()
    feature_df = pd.DataFrame({ "id":id })

    #down_time, up_time
    for column_name in ['down_time', 'up_time']:
        create_mean_value(df, feature_df, column_name)
        create_std_value(df, feature_df, column_name)
        create_max_value(df, feature_df, column_name)
        create_sum_value(df, feature_df, column_name)

    #length str
    for column_name in ['activity', 'down_event', 'up_event', 'text_change']:
        create_length_value(df, df, column_name)

    for column_name in ['length_activity', 'length_down_event', 'length_up_event', 'length_text_change']:
        create_mean_value(df, feature_df, column_name)
        create_std_value(df, feature_df, column_name)
        create_sum_value(df, feature_df, column_name)

    #+ - */

    column_pairs = [
        ('length_activity', 'length_down_event'),
        ('length_activity', 'length_up_event'),
        ('length_activity', 'length_text_change'),
        ('length_down_event', 'length_up_event'),
        ('length_down_event', 'length_text_change'),
        ('length_up_event', 'length_text_change'),
    ]

    for column_pair in column_pairs:
      added_column = add_columns(df, df, column_pair[0], column_pair[1])
      create_mean_value(df, feature_df, added_column)
      create_std_value(df, feature_df, added_column)
      create_sum_value(df, feature_df, added_column)

      diff_column = diff_columns(df, df, column_pair[0], column_pair[1])
      create_mean_value(df, feature_df, diff_column)
      create_std_value(df, feature_df, diff_column)
      create_sum_value(df, feature_df, diff_column)

      multiplied_column = multiply_columns(df, df, column_pair[0], column_pair[1])
      create_mean_value(df, feature_df, multiplied_column)
      create_std_value(df, feature_df, multiplied_column)
      create_sum_value(df, feature_df, multiplied_column)

      div_column = div_columns(df, df, column_pair[0], column_pair[1])
      create_mean_value(df, feature_df, div_column)
      create_std_value(df, feature_df, div_column)
      create_max_value(df, feature_df, div_column)
      create_sum_value(df, feature_df, div_column)


    #Extract 4 features.
    concat_columns(df, df, ['length_activity', 'length_down_event', 'length_up_event', 'length_text_change'], 'total_length')
    create_mean_value(df, feature_df, 'total_length')
    create_std_value(df, feature_df, 'total_length')
    create_max_value(df, feature_df, 'total_length')
    create_sum_value(df, feature_df, 'total_length')

    #cursion_position+word_count
    concat_columns(df, df, ['cursor_position', 'word_count'])
    create_mean_value(df, feature_df, 'cursor_position_word_count')
    create_std_value(df, feature_df, 'cursor_position_word_count')
    create_max_value(df, feature_df, 'cursor_position_word_count')

    #time_shift(free_time)
    gaps=[1, 2, 5, 7, 10, 14, 21, 30, 50]
    for gap in gaps:
        df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
        df[f'free_time{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']
        df.drop(f'up_time_shift{gap}', axis=1, inplace=True)

        create_mean_value(df, feature_df, f'free_time{gap}')
        create_std_value(df, feature_df,  f'free_time{gap}')
        create_max_value(df, feature_df, f'free_time{gap}')
        create_sum_value(df, feature_df, f'free_time{gap}')
        create_count_value(df, feature_df, f'free_time{gap}')

        # cursor position shift
        df[f'cursor_position_shift{gap}'] = df.groupby('id')['cursor_position'].shift(gap)
        df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[f'cursor_position_shift{gap}']
        df.drop(f'cursor_position_shift{gap}', axis=1, inplace=True)

        create_mean_value(df, feature_df, f'cursor_position_change{gap}')
        create_std_value(df, feature_df, f'cursor_position_change{gap}')
        create_max_value(df, feature_df, f'cursor_position_change{gap}')
        create_sum_value(df, feature_df, f'cursor_position_change{gap}')

        # word count shift
        df[f'word_count_shift{gap}'] = df.groupby('id')['word_count'].shift(gap)
        df[f'word_count_change{gap}'] = df['word_count'] - df[f'word_count_shift{gap}']
        df.drop(f'word_count_shift{gap}', axis=1, inplace=True)

        create_mean_value(df, feature_df, f'word_count_change{gap}')
        create_std_value(df, feature_df, f'word_count_change{gap}')
        create_max_value(df, feature_df, f'word_count_change{gap}')
        create_sum_value(df, feature_df, f'word_count_change{gap}')

        # cursor_position_word_count shift
        df[f'cursor_position_word_count_shift{gap}'] = df.groupby('id')['cursor_position_word_count'].shift(gap)
        df[f'cursor_position_word_count_change{gap}'] = df['cursor_position_word_count'] - df[f'cursor_position_word_count_shift{gap}']
        df.drop(f'cursor_position_word_count_shift{gap}', axis=1, inplace=True)

        create_mean_value(df, feature_df, f'cursor_position_word_count_change{gap}')
        create_std_value(df, feature_df, f'cursor_position_word_count_change{gap}')
        create_max_value(df, feature_df, f'cursor_position_word_count_change{gap}')
        create_sum_value(df, feature_df, f'cursor_position_word_count_change{gap}')

    #action_time
    create_mean_value(df, feature_df, 'action_time')
    create_std_value(df, feature_df, 'action_time')
    create_sum_value(df, feature_df, 'action_time')
    create_count_value(df, feature_df, 'action_time')

    feature_df['sum_time']=df['up_time'].groupby([df['id']]).max().values
    feature_df['count_action_time_percent']=feature_df['count_action_time']/feature_df['sum_action_time']
    feature_df['count_time_percent']=feature_df['count_action_time']/feature_df['sum_time']
    feature_df['active_percent']=feature_df['sum_action_time']/feature_df['sum_time']

    #cursor_position
    create_mean_value(df, feature_df, 'cursor_position')
    create_std_value(df, feature_df, 'cursor_position')
    create_max_value(df, feature_df, 'cursor_position')
    feature_df['cursor_speed']=feature_df['max_cursor_position']/feature_df['sum_action_time']

    #word_count
    create_mean_value(df, feature_df, 'word_count')
    create_std_value(df, feature_df, 'word_count')
    create_max_value(df, feature_df, 'word_count')
    feature_df['word_count_speed']=feature_df['max_word_count']/feature_df['sum_action_time']


    # concat 3 columns

    column_lists = [
        ('cursor_position', 'word_count', 'length_activity'),
        ('cursor_position', 'word_count', 'length_down_event'),
        ('cursor_position', 'word_count', 'length_text_change'),
        ('length_activity', 'length_down_event', 'length_text_change'),

    ]

    for columns in column_lists:
        concat_columns(df, df, columns)
        create_mean_value(df, feature_df, f'{columns[0]}_{columns[1]}_{columns[2]}')
        create_std_value(df, feature_df, f'{columns[0]}_{columns[1]}_{columns[2]}')
        create_max_value(df, feature_df, f'{columns[0]}_{columns[1]}_{columns[2]}')

    # Get activity
    activity = df['activity'].value_counts().keys().values

    # activity_i_count(bag of words model)
    for i in range(len(activity)):
        df[f'is_{activity[i]}'] = (df['activity'] == activity[i])
        feature_df[f'activity_{i}_count'] = df[f'is_{activity[i]}'].groupby([df['id']]).sum().values
        feature_df[f'activity_{i}_mean'] = df[f'is_{activity[i]}'].groupby([df['id']]).mean().values
        feature_df[f'activity_{i}_std'] = df[f'is_{activity[i]}'].groupby([df['id']]).std().values

    # Take the top 30 important ones
    event = df['down_event'].value_counts()[:30].keys().values

    # event (bag of words model)
    for i in range(len(event)):
        df[f'is_{event[i]}'] = (df['down_event'] == event[i])
        feature_df[f'event_{i}_count'] = df[f'is_{event[i]}'].groupby([df['id']]).sum().values
        feature_df[f'event_{i}_mean'] = df[f'is_{event[i]}'].groupby([df['id']]).mean().values
        feature_df[f'event_{i}_std'] = df[f'is_{event[i]}'].groupby([df['id']]).std().values

    # down_event The number of occurrences of each letter of
    for i in range(26):
        feature_df[f'event_letter_{i}'] = 0  # From a to z, case insensitive
        for j in range(len(activity)):
            word = activity[j].lower()  # For example input
            for k in word:
                if k == chr(97 + i):
                    feature_df[f'event_letter_{i}'] += feature_df[
                        f'event_{j}_count']

    # Take the top 15 important ones
    change = df['text_change'].value_counts()[:15].keys().values

    # text_change:bag of words model
    for i in range(len(change)):
        df[f'is_{change[i]}'] = (df['text_change'] == change[i])
        feature_df[f'change_{i}_count'] = df[f'is_{change[i]}'].groupby([df['id']]).sum().values
        feature_df[f'change_{i}_mean'] = df[f'is_{change[i]}'].groupby([df['id']]).mean().values
        feature_df[f'change_{i}_std'] = df[f'is_{change[i]}'].groupby([df['id']]).std().values

    # Classify features with obvious linear correlation.
    feature_df['count_action_time_grade2'] = feature_df['count_action_time'] // 500
    feature_df['max_cursor_grade2'] = feature_df['max_cursor_position'] // 300
    feature_df['max_word_count_grade2'] = feature_df['max_word_count'] // 50

    return feature_df

In [12]:
# Read data files from the public Kaggle competition!
train_logs = pd.read_csv(train_logs_file)
train_scores = pd.read_csv(train_scores_file)

# Strip position values for "Move from" activities
train_logs['activity'] = train_logs['activity'].apply(lambda x: "Move From" if str(x)[:9] == "Move From" else x)

# Call feature extraction function
# If the performance warning bothers you, check https://stackoverflow.com/questions/68292862/performancewarning-dataframe-is-highly-fragmented-this-is-usually-the-result-o
train_df = deal_df(train_logs)

# Merge X and y on id
train_df = pd.merge(train_df, train_scores, on="id", how="left")
train_df.drop(['id'], axis=1, inplace=True)
X = train_df.drop(['score'], axis=1)
y = train_df['score']

# Split up dataset;
# It is not allowed to use y_test!!! -- If you use it, you are disqualified from the assignment!!!
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Save  training set X, y  and test set X
# It is not allowed to use y_test!!! -- If you use it, you are disqualified from the assignment!!!
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

# Write data files for the private Kaggle competition!
X_train.to_csv(X_train_file, index_label="id")
y_train.to_csv(y_train_file, index_label="id")
X_test.to_csv(X_test_file, index_label="id")

C:\Users\nkalm\AppData\Local\Temp\ipykernel_6764\2612742836.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  target_df[f'std_{column_name}'] = source_df[column_name].groupby([source_df['id']]).std().values
C:\Users\nkalm\AppData\Local\Temp\ipykernel_6764\2612742836.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  target_df[f'max_{column_name}'] = source_df[column_name].groupby([source_df['id']]).max().values
C:\Users\nkalm\AppData\Local\Temp\ipykernel_6764\2612742836.py:14: PerformanceWarning: DataFrame is highly fragment

In [13]:
import sklearn.linear_model as LinearRegression
import sklearn.model_selection as model_selection

In [14]:
y_test_file = R"pc_y_test.csv"

In [15]:
X_train_baseline = pd.read_csv(X_train_file,index_col="id")
y_train_baseline = pd.read_csv(y_train_file,index_col="id")
X_test_baseline = pd.read_csv(X_test_file,index_col="id")

model = LinearRegression.LinearRegression()

model.fit(X_train_baseline, y_train_baseline)
y_test_baseline = model.predict(X_test_baseline)
pd.DataFrame(y_test_baseline, columns= ["score"]).to_csv(y_test_file, index_label="id")

In [18]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load the training data
X_train = pd.read_csv('pc_X_train.csv', index_col='id')
y_train = pd.read_csv('pc_y_train.csv', index_col='id')

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Load the test data
X_test = pd.read_csv('pc_X_test.csv', index_col='id')

# Predict the scores for the test data
y_test = model.predict(X_test)

# Save the predictions to a CSV file
pd.DataFrame(y_test, columns=['score'], index=X_test.index).to_csv('final_predictions.csv', index_label='id')



In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load the training data
x_train = pd.read_csv('pc_X_train.csv', index_col='id')
y_train = pd.read_csv('pc_y_train.csv', index_col='id')
x_test = pd.read_csv('pc_X_test.csv', index_col='id')

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

# Train a linear regression model
model = LinearRegression()
model.fit(x_train, y_train)

# Evaluate the model on the validation set
y_val_prediction = model.predict(x_val)
mse = mean_squared_error(y_val, y_val_prediction)
print(f"Validation MSE: {mse:.2f}")

# # Tune the model hyperparameters
# param_grid = {'fit_intercept': [True, False], 'copy_X': [True, False], 'positive': [True, False]}
# grid_search = GridSearchCV(model, param_grid, cv=5)
# grid_search.fit(x_train, y_train)
# print(f"Best hyperparameters: {grid_search.best_params_}")

# Train the final model on the entire training set using the best hyperparameters
# model = LinearRegression()
# model.fit(X_train, y_train)

# # Make predictions on the test set
y_prediction = model.predict(x_test)

# Save the predictions to a CSV file
pd.DataFrame(y_prediction, columns=['score']).to_csv('lineal_regression.csv', index_label='id')


Validation MSE: 1.29


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

# Load the training data
x_train = pd.read_csv('pc_X_train.csv', index_col='id')
y_train = pd.read_csv('pc_y_train.csv', index_col='id')
x_test = pd.read_csv('pc_X_test.csv', index_col='id')

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

# Train a lasso model
model = Lasso()
model.fit(x_train, y_train)

# Evaluate the model on the validation set
y_validation = model.predict(x_val)

mse = mean_squared_error(y_val, y_validation)
print(f"Validation MSE: {mse:.2f}")

y_prediction = model.predict(x_test)
pd.DataFrame(y_prediction, columns=['score']).to_csv('lasso.csv', index_label='id')

Validation MSE: 0.51


C:\Users\nkalm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.708e+02, tolerance: 1.713e-01
  model = cd_fast.enet_coordinate_descent(


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

# Load the datasets
x_train = pd.read_csv('pc_X_train.csv', index_col='id')
y_train = pd.read_csv('pc_y_train.csv', index_col='id')
x_test = pd.read_csv('pc_X_test.csv', index_col='id')

# Split the training data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

# Train an XGBoost model on the training set
model = XGBRegressor()
model.fit(x_train, y_train)

# Validate the model on the validation set
y_validation = model.predict(x_val)
mse = mean_squared_error(y_val, y_validation)
print(f"Validation MSE: {mse:.2f}")

# Predict the target variable for the test set
y_test_pred = model.predict(x_test)

# Save the predictions to a CSV file
pd.DataFrame(y_test_pred, columns=['score']).to_csv('xgboost_predictions.csv', index_label='id')

Validation MSE: 0.52
